In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [2]:
learning_rate = 0.01
batch_size=64
num_steps=500
display_step = 10

num_input = 784
num_output=10
dropout=0.60

X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_output])
keep = tf.placeholder(tf.float32)

In [35]:
def cnn_model(features, labels, mode):
    #Input layer
    ip_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
    
    #Convolutional layer 1
    conv_1 = tf.layers.conv2d(inputs=ip_layer, filters=32, padding='SAME', kernel_size=[5,5], activation=tf.nn.relu)
    
    #Polling layer 1
    pool_1 = tf.layers.max_pooling2d(inputs=conv_1, pool_size=[2,2], strides=2)
    
    #Convolutional layer 2
    conv_2 = tf.layers.conv2d(inputs = pool_1, filters=64, padding='same', kernel_size=[5,5], activation=tf.nn.relu)
    #pooling layer 2
    pool_2 = tf.layers.max_pooling2d(inputs=conv_2, pool_size=[2,2], strides=2)
    
    #feature map flattening
    pool_2_flat = tf.reshape(pool_2, [-1, 7*7*64])
    
    #dense layer that leads to classification
    dense_out = tf.layers.dense(inputs = pool_2_flat, units=1024, activation = tf.nn.relu)
    
    dropout = tf.layers.dropout(inputs = dense_out, rate=0.35, training=mode==tf.estimator.ModeKeys.TRAIN)
    
    #the final layers that performs classification
    
    fin_out = tf.layers.dense(inputs=dropout, units=10)
    
    
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=fin_out, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(fin_out, name="softmax_tensor")
       }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
      
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=fin_out)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
                   loss=loss,
                   global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
      }
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [36]:
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

In [37]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000290188A1390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [38]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [39]:
train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required|

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

In [40]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-1001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[0.09333838 0.05666564 0.15345679 0.13861327 0.09446464 0.09942152
  0.09564038 0.08407123 0.09340449 0.09092367]
 [0.11530642 0.09354988 0.12457045 0.12854078 0.08072734 0.06483972
  0.09198976 0.10573702 0.10257821 0.09216039]
 [0.09597792 0.12313122 0.10563761 0.10232124 0.09250984 0.07497267
  0.0894027  0.08680203 0.11289559 0.11634917]
 [0.08846299 0.06397145 0.10748336 0.13379093 0.08509973 0.08523707
  0.09462088 0.10960323 0.1350522  0.0966782 ]
 [0.08659209 0.11447373 0.08712254 0.12589106 0.08768743 0.08996585
  0.09090432 0.09908473 0.11396232

INFO:tensorflow:loss = 2.0459468, step = 1002
INFO:tensorflow:Saving checkpoints for 1002 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:Loss for final step: 2.0459468.


In [41]:
mnist_classifier.train(input_fn = train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-1002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1002 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:loss = 2.0867965, step = 1003
INFO:tensorflow:global_step/sec: 6.10414
INFO:tensorflow:loss = 0.9696405, step = 1103 (16.383 sec)
INFO:tensorflow:global_step/sec: 6.10451
INFO:tensorflow:loss = 0.45075548, step = 1203 (16.381 sec)
INFO:tensorflow:global_step/sec: 6.03833
INFO:tensorflow:loss = 0.4270109, step = 1303 (16.561 sec)
INFO:tensorflow:global_step/sec: 5.92412
INFO:tensorflow:loss = 0.4009849, step = 1403 (16.881 sec)
INFO:tensorflow:global_step/sec: 5.95372
INFO:tensorflow:loss = 0.29861978, step = 1503 (16.795 sec)
INFO:tensorflow:global_step/sec: 6.00942
INFO:

In [42]:
eval_fn = tf.estimator.inputs.numpy_input_fn(x={'x':eval_data}, y=eval_labels, num_epochs =1, shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn = eval_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-21T18:45:02Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-2002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-21-18:45:07
INFO:tensorflow:Saving dict for global step 2002: accuracy = 0.9494, global_step = 2002, loss = 0.17157558
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2002: /tmp/mnist_convnet_model\model.ckpt-2002
{'accuracy': 0.9494, 'loss': 0.17157558, 'global_step': 2002}
